<a href="https://colab.research.google.com/github/jqshang/CSC2541/blob/main/syn_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np

def syn_data(version="v1", num_acids=50, num_steps=20):
    np.random.seed(42)

    amino_acids = [f"A{i}" for i in range(1, num_acids + 1)]

    # dependency pairs Ai -> Aj
    pairs = [
        ("A1", "A16"),
        ("A16", "A17"),
        ("A17", "A30"),
        ("A30", "A36"),
        ("A30", "A16"),
        ("A36", "A50")
    ]

    # ----- Step 1: random initialize everything -----
    pos = np.random.uniform(-1, 1, size=(num_steps, num_acids, 3))
    ang = np.random.uniform(-180, 180, size=(num_steps, num_acids, 2))

    # helper: map acid → index
    idx = {a: i for i, a in enumerate(amino_acids)}

    # ----- Step 2: apply chosen version -----
    for t in range(num_steps - 1):
        for Ai, Aj in pairs:
            i = idx[Ai]
            j = idx[Aj]

            Ai_pos = pos[t, i]
            Ai_ang = ang[t, i]
            Aj_pos_prev = pos[t, j]
            Aj_ang_prev = ang[t, j]

            if version == "v1":
                # copy previous
                pos[t+1, j] = Aj_pos_prev.copy()
                ang[t+1, j] = Aj_ang_prev.copy()
                # update
                pos[t+1, j] += Aj_pos_prev * np.sin(Ai_pos)
                ang[t+1, j] -= Ai_ang

            elif version == "v2":
                # angles copy only
                ang[t+1, j] = Aj_ang_prev.copy()
                # update
                pos[t+1, j] = Aj_pos_prev * np.sin(Ai_pos)
                ang[t+1, j] -= Ai_ang

            elif version == "v3":
                # copy previous
                pos[t+1, j] = Aj_pos_prev.copy()
                ang[t+1, j] = Aj_ang_prev.copy()
                # update
                pos[t+1, j] += np.sin(Ai_pos)
                ang[t+1, j] -= Ai_ang

            else:
                raise ValueError("version must be 'v1', 'v2', or 'v3'")

    return pos, ang


In [29]:
p, a = syn_data("v2")
p

array([[[-2.50919762e-01,  9.01428613e-01,  4.63987884e-01],
        [ 1.97316968e-01, -6.87962719e-01, -6.88010959e-01],
        [-8.83832776e-01,  7.32352292e-01,  2.02230023e-01],
        ...,
        [-4.96435408e-01, -5.50298822e-03, -3.98243380e-01],
        [-4.30319011e-01, -9.26226105e-01,  2.19128668e-01],
        [ 5.35804646e-03, -8.97042498e-01, -4.42707072e-01]],

       [[ 8.16531772e-01, -5.20876219e-01, -7.10210256e-01],
        [-2.10944794e-02,  9.71300908e-01, -5.15889457e-01],
        [ 3.44271095e-01,  5.23239231e-01, -5.24724912e-01],
        ...,
        [-9.25303623e-01,  6.45201121e-01, -2.79618717e-01],
        [-7.45878975e-01,  4.44865201e-02,  5.39987106e-01],
        [-2.57543779e-03,  1.59921191e-01, -2.16545275e-01]],

       [[-8.96636558e-01,  6.27092631e-02,  8.12702432e-02],
        [ 2.74859803e-01,  4.52182667e-01,  9.51704159e-01],
        [ 3.26006966e-02, -3.54087054e-01,  5.90372390e-01],
        ...,
        [ 9.73279157e-01, -2.51458408e-01,

##Ignore below:

In [14]:
import numpy as np
import pandas as pd

np.random.seed(42)

num_acids = 50
num_steps = 20
amino_acids = [f"A{i}" for i in range(1, num_acids + 1)]

# dependency pairs Ai -> Aj
pairs = [
    ("A1", "A16"),
    ("A16", "A17"),
    ("A17", "A30"),
    ("A30", "A36"),
    ("A30", "A16"),
    ("A36", "A50")
]

# amino acids affected by rules (targets only)
dependent_targets = {"A16", "A17", "A30", "A36", "A50"}

# ----- Step 1: Generate full random dataset for all acids and all times -----
data = pd.DataFrame(index=range(num_steps), columns=amino_acids)

for t in range(num_steps):
    for a in amino_acids:
        pos = np.random.uniform(-1, 1, size=3)
        angles = np.random.uniform(-180, 180, size=2)
        data.at[t, a] = {"pos": pos, "angles": angles}




In [15]:
# ----- Step 2: Overwrite dependent amino acids using the rules -----
# ----- v1
for t in range(num_steps - 1):
    for Ai, Aj in pairs:
        Ai_prev = data.at[t, Ai]
        Aj_prev = data.at[t, Aj]
        Aj_next = data.at[t+1, Aj]

        # overwrite only Aj (the target)
        # step 1: copy previous values
        Aj_next["pos"] = Aj_prev["pos"].copy()
        Aj_next["angles"] = Aj_prev["angles"].copy()

        # step 2: apply update rule
        Aj_next["pos"] += Aj_prev["pos"] * np.sin(Ai_prev["pos"])
        Aj_next["angles"] -= Ai_prev["angles"]

In [9]:
# ----- Step 2: Overwrite dependent amino acids using the rules -----
# ----- v2
for t in range(num_steps - 1):
    for Ai, Aj in pairs:
        Ai_prev = data.at[t, Ai]
        Aj_prev = data.at[t, Aj]
        Aj_next = data.at[t+1, Aj]

        # overwrite only Aj (the target)
        # step 1: copy previous values
        # Aj_next["pos"] = Aj_prev["pos"].copy()
        Aj_next["angles"] = Aj_prev["angles"].copy()

        # step 2: apply update rule
        Aj_next["pos"] = Aj_prev["pos"] * np.sin(Ai_prev["pos"])
        Aj_next["angles"] -= Ai_prev["angles"]

In [12]:
# ----- Step 2: Overwrite dependent amino acids using the rules -----
# ----- v3
for t in range(num_steps - 1):
    for Ai, Aj in pairs:
        Ai_prev = data.at[t, Ai]
        Aj_prev = data.at[t, Aj]
        Aj_next = data.at[t+1, Aj]

        # overwrite only Aj (the target)
        # step 1: copy previous values
        Aj_next["pos"] = Aj_prev["pos"].copy()
        Aj_next["angles"] = Aj_prev["angles"].copy()

        # step 2: apply update rule
        Aj_next["pos"] += np.sin(Ai_prev["pos"])
        Aj_next["angles"] -= Ai_prev["angles"]

In [16]:
data
# data.at[5, "A16"]


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A41,A42,A43,A44,A45,A46,A47,A48,A49,A50
0,"{'pos': [-0.250919762305275, 0.901428612819832...","{'pos': [-0.6880109593275947, -0.8838327756636...","{'pos': [-0.9588310114083951, 0.93981970432398...","{'pos': [-0.6331909802931324, -0.3915155140809...","{'pos': [0.22370578944475894, -0.7210122786959...","{'pos': [0.5703519227860272, -0.60065243568328...","{'pos': [0.21508970380287673, -0.6589517526254...","{'pos': [0.6167946962329223, -0.39077246165325...","{'pos': [-0.7559235303104423, -0.0096461797774...","{'pos': [0.32504456870796394, -0.3765778478211...",...,"{'pos': [0.2840632923085755, -0.83172007000990...","{'pos': [-0.9816058967667407, -0.7970569142679...","{'pos': [0.09746757873317224, 0.38379039538538...","{'pos': [-0.5255018250063999, -0.3492006036814...","{'pos': [0.31522578460068673, 0.13661720667094...","{'pos': [-0.5120207132418328, 0.94602110950489...","{'pos': [0.5896226070832968, 0.005274186210384...","{'pos': [0.44490423052301065, -0.4384552751182...","{'pos': [0.8809171687058286, 0.907857154005174...","{'pos': [0.8566371251754508, -0.14363170336537..."
1,"{'pos': [-0.4111022158608286, -0.2298045427961...","{'pos': [0.11360252491670031, 0.87230954832156...","{'pos': [0.23001445339833948, 0.98010770020852...","{'pos': [0.4815372355084089, 0.394031481990535...","{'pos': [0.6187223109570272, 0.620226789358361...","{'pos': [0.0030325893743992083, 0.596590357933...","{'pos': [0.7800106836351326, -0.32400968629692...","{'pos': [-0.9281154524065158, -0.0688039637350...","{'pos': [-0.9389995001219011, -0.9253036225015...","{'pos': [0.04448652010960874, 0.53998710619722...",...,"{'pos': [0.5542938318548736, 0.116808499471610...","{'pos': [-0.0147497914182817, -0.9772927104651...","{'pos': [-0.7649475064457902, 0.29842060423212...","{'pos': [-0.2502588409525919, -0.4285758274362...","{'pos': [-0.9756910506203673, 0.93975765341527...","{'pos': [0.9859295922386007, -0.85240687052920...","{'pos': [0.25879727627052507, 0.39149737796923...","{'pos': [0.802316020981978, -0.909107239317084...","{'pos': [-0.08868649442857413, 0.2402651956030...","{'pos': [0.3373482650705697, -0.19715771003234..."
2,"{'pos': [0.3963234280394903, 0.072192732688240...","{'pos': [-0.6747661213102174, 0.82185436898768...","{'pos': [0.22683039187157972, -0.1635139274187...","{'pos': [-0.947266051005496, -0.24707326624390...","{'pos': [0.18826143070427026, -0.2382182867379...","{'pos': [-0.06261368041005944, -0.170360995324...","{'pos': [0.6258020182601551, 0.999435346572261...","{'pos': [0.889531459764856, 0.6992947813548229...","{'pos': [0.9081020545174447, 0.212349268901760...","{'pos': [-0.283674563934319, -0.77288481560074...",...,"{'pos': [0.06517886510317172, -0.8963529263551...","{'pos': [0.9799204647798905, -0.35529231005055...","{'pos': [0.5204557197793731, 0.191277481215688...","{'pos': [0.8590582884956517, 0.661238815575458...","{'pos': [0.8766809136420757, -0.63753386766867...","{'pos': [0.6836575535165441, -0.72045524674742...","{'pos': [-0.6714684041380141, 0.62914944046276...","{'pos': [0.7544010816262166, -0.21510978515472...","{'pos': [-0.07464042866078713, -0.397244251671...","{'pos': [0.29339180634535406, -0.2855478556087..."
3,"{'pos': [-0.7662039185832719, 0.87966424722695...","{'pos': [0.588050378540592, 0.2401455118570270...","{'pos': [-0.6966502405344976, -0.3765558644089...","{'pos': [0.13977936974263305, 0.52491737148138...","{'pos': [-0.7787365260895855, 0.69290458346903...","{'pos': [-0.7001651453024524, -0.5414972095347...","{'pos': [0.38789688893420005, 0.08544888669519...","{'pos': [0.8169011226672567, 0.166783589532240...","{'pos': [-0.6932971937678396, 0.17245966403359...","{'pos': [0.744247817888303, 0.8642365649672248...",...,"{'pos': [0.20103372086730675, 0.33007334909251...","{'pos': [-0.2337229435010051, 0.03783541056567...","{'pos': [-0.8344026641497471, 0.20630421893277...","{'pos': [-0.28865456707010173, 0.4380918103684...","{'pos': [0.3273423307252963, 0.873659478649516...",